In [1]:
%load_ext autoreload
%autoreload 2 
%reload_ext autoreload
%matplotlib inline
import matplotlib.pyplot as plt

# always import gbm_algos first !
import xgboost, lightgbm, catboost
from gplearn.genetic import SymbolicRegressor

# To access the contents of the parent dir
import sys; sys.path.insert(0, '../')
import os
from scipy.io import loadmat
from utils import *
from preprocess import *
from models import RobustPCANN

# Let's do facy optimizers
from optimizers import Lookahead, AdamGC, SGDGC
from madgrad import MADGRAD
from lbfgsnew import LBFGSNew

from pytorch_robust_pca import *

# Modify at /usr/local/lib/python3.9/site-packages/torch_lr_finder/lr_finder.py
from torch_lr_finder import LRFinder

# Tracking
from tqdm import trange

import sympy
import sympytorch

Running Python 3.9.6
You can use npar for np.array


In [2]:
# Loading the KS sol
DATA_PATH = "../deephpms_data/KS_simple3.pkl"
data = pickle_load(DATA_PATH)
t = data['t']
x = data['x']
X, T = np.meshgrid(x, t)
Exact = data['u'].T

# Adding noise
noise_intensity = 1
noisy_xt = True

x_star = X.flatten()[:,None]
t_star = T.flatten()[:,None]
X_star = np.hstack((x_star, t_star))
u_star = Exact.T.flatten()[:,None]

if noisy_xt: 
    print("Noisy (x, t)")
    X_star = perturb(X_star, intensity=noise_intensity, noise_type="normal")
else: print("Clean (x, t)")

# Doman bounds
lb = X_star.min(axis=0)
ub = X_star.max(axis=0)

N = 5000
print(f"Fine-tuning with {N} samples")
idx = np.random.choice(X_star.shape[0], N, replace=False)
X_u_train = X_star[idx, :]
u_train = u_star[idx,:]
u_train_clean = u_train
noise = perturb(u_train, intensity=noise_intensity, noise_type="normal", overwrite=False)
u_train = u_train + noise
print("Perturbed Exact with intensity =", float(noise_intensity))

# Robust PCA
print("Running Robust PCA on X_u_train")
rpca = R_pca_numpy(X_u_train)
X_train_L, X_train_S = rpca.fit(tol=1e-20, max_iter=25000, iter_print=100, verbose=False)
print('Robust PCA Loss:', mean_squared_error(X_u_train, X_train_L+X_train_S))
X_train_S = to_tensor(X_train_S, False)

print("Running Robust PCA on X_u_train")
rpca = R_pca_numpy(u_train)
u_train_L, u_train_S = rpca.fit(tol=1e-20, max_iter=25000, iter_print=100, verbose=False)
print('Robust PCA Loss:', mean_squared_error(u_train, u_train_L+u_train_S))
u_train_S = to_tensor(u_train_S, False)

# del rpca, X_train_L, u_train_L, X_star, u_star, Exact, data

# Convert to torch.tensor
X_u_train = to_tensor(X_u_train, True)
u_train = to_tensor(u_train, False)

# lb and ub are used in adversarial training
scaling_factor = 1.0
lb = scaling_factor*to_tensor(lb, False)
ub = scaling_factor*to_tensor(ub, False)

# Feature names, base on the symbolic regression results
feature_names=('uf', 'u_x', 'u_xx', 'u_xxxx'); feature2index = {}

Loaded from ../deephpms_data/KS_simple3.pkl
Noisy (x, t)
Fine-tuning with 5000 samples
Perturbed Exact with intensity = 1.0
Running Robust PCA on X_u_train
Robust PCA Loss: 2.871306478090326e-32
Running Robust PCA on X_u_train
Robust PCA Loss: 5.934679067485165e-34


In [4]:
((noise-u_train_S.numpy())**2).mean()

0.801254001922349

In [5]:
out, PSD = fft1d_denoise(u_train, thres=None, c=0)

In [6]:
((noise-(u_train-out).numpy())**2).mean()

0.7683256645290987